# Code Generator


In [2]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess


In [1]:
print("Hello, World!")

Hello, World!


In [3]:
load_dotenv(override=True)

groq_api_key = os.getenv('GROQ_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

if openai_api_key:
    print(f"OpenAI API Key is set")
else:
    print("OpenAI API Key not set (optional; add OPENAI_API_KEY to .env to use OpenAI)")

Groq API Key exists and begins gsk_


In [5]:
# Connect to client libraries

groq_url = "https://api.groq.com/openai/v1"
ollama_url = "http://localhost:11434/v1"

# OpenAI client only if OPENAI_API_KEY is set
openai = OpenAI(api_key=openai_api_key) if openai_api_key else None
groq = OpenAI(api_key=groq_api_key, base_url=groq_url) if groq_api_key else None
ollama = OpenAI(api_key="ollama", base_url=ollama_url)


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable